# Notebook I: Morse Feature Generation Walkthrough

This notebook is a walkthrough of how to generate the Morse feature vector of one molecule. It follows the **MORSE** procedure: 

1. [**M**odify the point cloud](#modify-the-point-cloud);
2. [**O**btain the triangulation](#obtain-the-triangulation);
3. [**R**ealise the Morse transform](#realise-the-morse-transform);
4. [**S**upplement with chemical information](#supplement-with-chemical-information); and
5. [**E**ncapsulate in a vector](#encapsulate-in-a-vector).

Note that this notebook is purely illustrative and generating large batches of Morse features should be done via a dedicated script (see the next notebook in the series).

# Packages
Import the Python packages necessary to run the notebook.

In [1]:
import numpy as np
import pandas as pd

Add the path of the `src` directory.

In [ ]:
from pathlib import Path

# The path of the directory which contains the src directory
src_dir = Path('../src')

# Add the path to system path
import sys
try:
  sys.path.index(str(src_dir))
except ValueError:
  sys.path.insert(0,str(src_dir))

# Print system path
sys.path

Import classes from the `magic` module. (MAGIC: Morse Augmented Geometric Invariant Classification.)

In [3]:
from magic import SdfReader, MoleculeComplex, DataAlignment, DataAugmentation, Baseline

Import classes from the `classification` module.

In [ ]:
from classification import HelperFunctions

# **MORSE**

## **M**odify the point cloud 

### Read SDF files

Read the SDF file of a ligand of the DUD-E protein target CXCR4 into a pandas dataframe and mean-centre the co&ouml;rdinates.

In [5]:
# Any '!!! Warning !!! Distance between atoms X and Y (Z A) is suspicious.' may be ignored.
reader = SdfReader('../data/sdf/dude/', 'cxcr4')
file_path = reader.sdf_folder + 'actives_final.lowE.prepped.sdf'
molecule_id = 10 # Feel free to change the molecule ID
unaligned_df = reader._reading_one_sdf_file(file_path, molecule_id)

View the molecular point cloud data frame:

In [6]:
display(unaligned_df[['symbol', 'x', 'y', 'z']])

,symbol,x,y,z
0,C,3.280203,-1.756306,1.410094
1,C,2.819703,-0.903506,0.259694
2,C,1.537903,-0.347506,0.287094
3,N,1.091703,0.446094,-0.734506
4,C,-0.279097,0.996694,-0.692006
5,C,-1.281597,-0.007806,-1.231106
6,C,-2.532597,0.337894,-0.476806
7,N,-3.307597,1.325994,-1.224506
8,N,-4.476197,1.106494,-1.545206
9,N,-5.643397,0.887494,-1.865706


### Point cloud alignment

Align the point cloud co&ouml;rdinates with the principal component axes.

In [7]:
al = DataAlignment()
aligned_df = al._alignSingle(unaligned_df)

View the aligned dataframe:

In [8]:
display(aligned_df[['symbol', 'x', 'y', 'z']])

,symbol,x,y,z
0,C,3.574691,1.683303,-0.469879
1,C,2.947667,0.358733,-0.130610
2,C,1.572298,0.290972,0.107438
3,N,0.970900,-0.894022,0.434043
4,C,-0.490071,-0.940798,0.651840
5,C,-1.227767,-1.099023,-0.665516
6,C,-2.535464,-0.413162,-0.395328
7,N,-3.505410,-1.376689,0.121239
8,N,-4.587837,-1.551412,-0.439544
9,N,-5.669019,-1.726172,-0.999493


## **O**btain the triangulation

Generate the molecular complex from the molecular point cloud. The molecular complex is the _pruned Delaunay triangulation_ of the point cloud with weights corresponding to the van der Waals radii of the constituent atoms.

In [9]:
m = MoleculeComplex()
simplices = m.build(aligned_df)

print('The molecular complex contains', sum([len(s) for s in m.simplices]), 'simplices:', '\n\n',
      len(m.zero_simplices), '0-simplices:', m.zero_simplices, '\n\n',
      len(m.one_simplices), '1-simplices:', m.one_simplices, '\n\n',
      len(m.two_simplices), '2-simplices:', m.two_simplices, '\n\n',
      len(m.three_simplices), '3-simplices:', m.three_simplices, '\n\n',
      'Dimension of complex =', m.dim_complex)

The molecular complex contains 241 simplices: 

 32 0-simplices: [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31]] 

 101 1-simplices: [[12, 28], [2, 3], [10, 28], [10, 13], [8, 7], [27, 7], [13, 5], [3, 4], [26, 6], [16, 17], [0, 2], [0, 19], [16, 15], [2, 4], [24, 5], [4, 7], [3, 23], [0, 18], [6, 7], [19, 21], [17, 18], [3, 14], [10, 12], [10, 5], [26, 5], [13, 22], [25, 6], [2, 5], [4, 23], [12, 13], [11, 30], [10, 4], [4, 5], [11, 29], [24, 7], [0, 22], [0, 21], [29, 6], [10, 11], [27, 11], [11, 5], [4, 13], [5, 23], [23, 15], [24, 25], [3, 5], [17, 14], [0, 20], [14, 31], [31, 15], [24, 6], [2, 14], [12, 30], [5, 7], [13, 23], [10, 6], [4, 15], [11, 12], [14, 23], [0, 17], [26, 11], [14, 15], [2, 13], [17, 31], [8, 26], [17, 2], [12, 29], [18, 19], [10, 29], [1, 20], [0, 1], [16, 31], [27, 6], [1, 17], [8, 6], [4, 14], [16, 18], [20, 21], [19, 20], 

Visualise the 2-skeleton of the molecular complex. You may need to change the `renderer` keyword argument.

In [34]:
# Click and drag the Plotly figure to rotate the molecular complex!
m.plot_skeleton(n=2, renderer='notebook_connected')

Compare the orientation of the above molecular complex with the complex generated from a randomly rotated point cloud:

In [33]:
# Generate a randomly rotated point cloud
a = DataAugmentation()
rand_rot_df_list = a.augment_single([unaligned_df], N_aug=1)
rand_rot_df = rand_rot_df_list[0]

# Generate and display the molecular complex
mm = MoleculeComplex()
mm.build(rand_rot_df)
mm.plot_skeleton(n=2, renderer='notebook_connected')

## **R**ealise the Morse transform

### One direction

Calculate the Morse transform of depth 10 of the molecular complex for the single direction t = (1, 0, 0).

In [12]:
# The query direction
t = np.array( [1, 0, 0] ) 

# Generate the padded Morse transform vector for depth 10
_, _, padded_output = m.getCriticalPoints(t, top=10)

def morse_data(padded_vector_output, top=10):
  """
  A function to recover the Morse data matrix from 
  the zero-padded flattened Morse transform vector.
  """
  raw_matrix = padded_vector_output.reshape((top, 14), order='F')  

  # Filter out the zero-padded entries 
  # and non-relevant columns
  zero_mask = raw_matrix == 0
  zero_mask = np.all(zero_mask, axis=1)
  critical_mask = ~zero_mask
  morse_data = raw_matrix[critical_mask, 0:4]

  # Format the Morse data in a pandas dataframe
  morse_data_df = pd.DataFrame({
                                "Projected height": morse_data[:, 0], 
                                "Reduced Betti -1": pd.Series(morse_data[:, 1], dtype='int'), 
                                "Reduced Betti 0": pd.Series(morse_data[:, 2], dtype='int'), 
                                "Reduced Betti 1": pd.Series(morse_data[:, 3], dtype='int')
                              })
  return morse_data_df

morse_data_df = morse_data(padded_output, top=10)

# Display the dataframe
print('direction =', t)
display(morse_data_df.style.hide())

direction = [1 0 0]


Projected height,Reduced Betti -1,Reduced Betti 0,Reduced Betti 1
4.933750,1,0,0
-1.276329,1,0,0
-1.591381,1,0,0
-1.692579,0,2,0
-2.048774,0,1,0
-2.172702,0,0,1


Visualise the critical vertices. The critical index $(\widetilde{\beta}_{-1}, \,\widetilde{\beta}_{0}, \,\widetilde{\beta}_{1})$ of a vertex is the list of the reduced Betti numbers of the upper link of a vertex. Vertices which have a critical index of (0, 0, 0) are regular and not critical.

In [35]:
# You can click on the Plotly legend to highlight certain vertices by critical index
m.plot_crit_pt(two_skeleton=True, renderer='notebook_connected')

Calculate the Morse transform for another direction t = (1, 1, 1) and note the change in the Morse data. (The number of critical vertices, critical values and critical indices can change.)

In [36]:
# The query direction
t = np.array( [1, 1, 1] ) # try [1, 0, 1] too

# Generate the padded Morse transform vector for depth 10
_, _, padded_vector_output = m.getCriticalPoints(t, top=10)
morse_data_df = morse_data(padded_vector_output, top=10)

# Display the dataframe
print('direction =', t)
display(morse_data_df.style.hide())

# Visualise the critical vertices
m.plot_crit_pt(two_skeleton=True, renderer='notebook_connected')

direction = [1 1 1]


Projected height,Reduced Betti -1,Reduced Betti 0,Reduced Betti 1
3.558591,1,0,0
1.676111,1,0,0
0.333855,0,1,0
-0.494191,0,1,0
-0.505722,0,0,1


### Multiple directions

Calculate the Morse transform for a set of directions.

In [15]:
# Randomly generate 5 directions from the surface of the 2-sphere.
n_directions = 5
directions = m.getDirectionVectors(N=n_directions, method='inverseTransform', random_seed=None)

# Or manually choose the directions 
# e.g.
# directions = [
#     [1, 0, 0],
#     [1, 1, 1],
#     [0, 1, 0],
#     [0, 0, 1],
#     [1, 0, 1]
# ]
# n_directions = len(directions)

print(directions)

[[-0.59572694 -0.61924168 -0.51151652]
 [ 0.20176728 -0.43507067 -0.87749842]
 [-0.87670302 -0.32010679 -0.35905912]
 [-0.23918202 -0.19658759 -0.95086554]
 [-0.81296885  0.57668195  0.08074388]]


In [16]:
# Generate the padded Morse data vector for depth 10 for 5 directions
depth = 10
padded_vector_output = m.getFeature(directions, top=depth, parallel_calc=False)

# Reshape the padded vector to a matrix, where each row corresponds to the 
# flattened padded Morse transform for each direction
# (it's padded because there are a different number of critical vertices for each direction)
size_one_direction = padded_vector_output.shape[0] / n_directions
size_one_direction = int(size_one_direction )

padded_matrix_output = padded_vector_output.reshape((n_directions, size_one_direction), order='C')

# Display the Morse transform output for each direction
for idx, single_direction_output in enumerate(padded_matrix_output):
    morse_data_df = morse_data(single_direction_output, top=depth)

    # Display the More data matrix dataframe
    print('direction', idx, '=', directions[idx])
    display(morse_data_df.style.hide())
    print('\n')

direction 0 = [-0.59572694 -0.61924168 -0.51151652]


Projected height,Reduced Betti -1,Reduced Betti 0,Reduced Betti 1
4.957362,1,0,0




direction 1 = [ 0.20176728 -0.43507067 -0.87749842]


Projected height,Reduced Betti -1,Reduced Betti 0,Reduced Betti 1
1.725293,1,0,0
1.501383,1,0,0
0.941975,1,0,0
0.812400,0,1,0
0.804094,0,1,0
0.553279,0,1,0
0.484238,1,0,0
0.401218,0,0,1
0.134998,0,1,0
-1.309052,0,1,0




direction 2 = [-0.87670302 -0.32010679 -0.35905912]


Projected height,Reduced Betti -1,Reduced Betti 0,Reduced Betti 1
5.881483,1,0,0




direction 3 = [-0.23918202 -0.19658759 -0.95086554]


Projected height,Reduced Betti -1,Reduced Betti 0,Reduced Betti 1
2.645655,1,0,0
1.944492,1,0,0
1.820264,0,1,0
1.684008,1,0,0
1.142530,0,1,0
0.556889,0,1,0
0.170555,1,0,0
-0.017315,0,0,1
-0.651359,0,1,0




direction 4 = [-0.81296885  0.57668195  0.08074388]


Projected height,Reduced Betti -1,Reduced Betti 0,Reduced Betti 1
3.775609,1,0,0
3.541784,1,0,0
3.532581,1,0,0
3.148939,0,1,0
2.885059,1,0,0
2.839453,0,1,0
2.309670,0,1,0
2.065667,0,1,0
1.791070,0,0,1
-1.353846,1,0,0


Flatten the Morse transform output to a padded pandas dataframe.

In [17]:
# The dimensional reduction in this cell is required due to a legacy 
# artefact of the code implementation. Specifically, the method
# MoleculeComplex.getFeature() does not just compute the Morse transform 
# (as described in the paper), but also records the atom-wise chemical 
# information associated with each critical vertex. The dimensional 
# reduction here strips the extraneous chemical information to give a 
# 'pure' padded Morse transform output.
padded_vector_output_df = pd.DataFrame([padded_vector_output])

h = HelperFunctions()
dcc = [0,1,2]
depth = 10

morse_transform_output = h.dim_reduction(
                                            padded_vector_output_df, depth,
                                            depth,
                                            n_directions,
                                            crit_categories=13,
                                            desired_crit_categories=dcc,
                                            projected_heights=True
                                        )

display(morse_transform_output)

,0,1,2,3,4,5,6,7,8,9,...,590,591,592,593,594,595,596,597,598,599
0,4.957362,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## **S**upplement with chemical information 
Extract the various chemical properties of the molecule.

In [18]:
# n.b. any '!!! Warning !!! Distance between atoms X and Y (Z A) is suspicious.' may be ignored.
reader = Baseline('../data/sdf/dude/', 'cxcr4')
file_path = reader.sdf_folder + 'actives_final.lowE.prepped.sdf'
chemical_properties = reader._reading_one_sdf_file(file_path, molecule_id)
chemical_properties_df = pd.DataFrame([chemical_properties])

This process extracts more chemical information than required. We shall restrict it to the three properties (partial charge, lipophilicity and molar refractivity) relevant to the Morse feature vector. N.B. We automatically extract the quantiles of those three chemical properties in the same step.

In [19]:
# Dimensionally reduce the chemical properties
h = HelperFunctions()
chem_quantiles_df = h.baseline_dim_reduction(chemical_properties_df,
                                n_original_properties=17,
                                desired_properties=[3,4,5],
                                num_quantiles=9
                                )
display(chem_quantiles_df)

chem_quantiles = chem_quantiles_df.to_numpy()

Columns selected: [3, 4, 5, 20, 21, 22, 37, 38, 39, 54, 55, 56, 71, 72, 73, 88, 89, 90, 105, 106, 107, 122, 123, 124, 139, 140, 141]


,3,4,5,20,21,22,37,38,39,54,...,90,105,106,107,122,123,124,139,140,141
0,0.69,0.2887,3.509,0.49009,0.18312,3.3285,0.154,0.138025,2.8485,0.08,...,1.0682,-0.2125,-0.2039,1.057,-0.56351,-0.32044,1.057,-0.68,-0.8186,0.8238


## **E**ncapsulate in a vector

Calculate the quantiles of the Morse transform output.

In [20]:
# Also add the quantiles of the distribution of the number of critical vertices
# for each direciton.
morse_quantiles, _ = h.summary(morse_transform_output,
                                depth,
                                n_directions,
                                crit_categories=len(dcc),
                                integer_categories=[0,1,2],
                                projected_heights=True,
                                add_num_crit_pts=True,
                                add_euler_char=False,
                                sum_over_all_crit_pts=True,
                                bin_count_betti=False,
                                pure_ect=False,
                                moments=False
                               )

Started quantile calculation. . .
Finished quantile calculation!



Concatenate the Morse and chemical quantiles together to form the Morse feature vector.

In [21]:
# morse_features = np.concatenate((morse_quantiles, chem_quantiles), axis=1)

morse_quantiles_matrix = morse_quantiles.reshape( (9, 5) )
chem_quantiles_matrix = chem_quantiles.reshape( (9, 3) )

morse_feature_matrix = np.concatenate((morse_quantiles_matrix, chem_quantiles_matrix), axis=1)
morse_feature_vector = morse_feature_matrix.flatten(order='F')

print('A Morse feature vector of length', len(morse_feature_vector))

A Morse feature vector of length 72


Visualise the constituent parts of the Morse feature vector.

In [22]:
# Format the Morse feature vector as a dataframe 
# (The 'Percentiles' column is not an explicit part of the Morse feature vector.)
morse_feature_df = pd.DataFrame({
                            "Percentiles": pd.Series([100, 90, 75, 60, 50, 40, 25, 10, 0], dtype='int'), 
                            "Projected height": morse_quantiles_matrix[:, 0], 
                            "Reduced Betti -1": pd.Series(morse_quantiles_matrix[:, 1], dtype='int'), 
                            "Reduced Betti 0": pd.Series(morse_quantiles_matrix[:, 2], dtype='int'), 
                            "Reduced Betti 1": pd.Series(morse_quantiles_matrix[:, 3], dtype='int'),
                            "# Critical Vertices": pd.Series(morse_quantiles_matrix[:, 4], dtype='int'),
                            "Partial Charge": pd.Series(chem_quantiles_matrix[:, 0], dtype='float'), 
                            "Lipophilicity": pd.Series(chem_quantiles_matrix[:, 1], dtype='float'),
                            "Polarizability": pd.Series(chem_quantiles_matrix[:, 2], dtype='float'),
                            })

# Display the dataframe
display(morse_feature_df.style.hide())

Percentiles,Projected height,Reduced Betti -1,Reduced Betti 0,Reduced Betti 1,# Critical Vertices,Partial Charge,Lipophilicity,Polarizability
100,5.881483,1,1,1,10,0.690000,0.288700,3.509000
90,3.541784,1,1,0,10,0.490090,0.183120,3.328500
75,2.645654,1,1,0,10,0.154000,0.138025,2.848500
60,1.820264,1,1,0,9,0.080000,0.123000,2.487400
50,1.684008,0,0,0,9,0.080000,0.123000,2.202000
40,0.941975,0,0,0,9,0.080000,0.123000,1.068200
25,0.553279,0,0,0,1,-0.212500,-0.203900,1.057000
10,-0.017315,0,0,0,1,-0.563510,-0.320440,1.057000
0,-1.353846,0,0,0,1,-0.680000,-0.818600,0.823800


# Next notebook
In the next notebook in the series (Notebook II) you shall see how to generate large batches of Morse data. . .